In [9]:
import torch
from torchvision import models


In [10]:
!pwd

/home/ning/Repo/kubeml/myExperiments


In [11]:
model0 = models.resnet18(pretrained= False)
model1 = models.resnet18(pretrained= False)

model0.load_state_dict(torch.load(f"./model_0.pt"))
model1.load_state_dict(torch.load(f"./model_1.pt"))

print(dir(model0)) 

RuntimeError: Error(s) in loading state_dict for ResNet:
	Unexpected key(s) in state_dict: "layer1.2.conv1.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.bn1.running_mean", "layer1.2.bn1.running_var", "layer1.2.bn1.num_batches_tracked", "layer1.2.conv2.weight", "layer1.2.bn2.weight", "layer1.2.bn2.bias", "layer1.2.bn2.running_mean", "layer1.2.bn2.running_var", "layer1.2.bn2.num_batches_tracked", "layer2.2.conv1.weight", "layer2.2.bn1.weight", "layer2.2.bn1.bias", "layer2.2.bn1.running_mean", "layer2.2.bn1.running_var", "layer2.2.bn1.num_batches_tracked", "layer2.2.conv2.weight", "layer2.2.bn2.weight", "layer2.2.bn2.bias", "layer2.2.bn2.running_mean", "layer2.2.bn2.running_var", "layer2.2.bn2.num_batches_tracked", "layer2.3.conv1.weight", "layer2.3.bn1.weight", "layer2.3.bn1.bias", "layer2.3.bn1.running_mean", "layer2.3.bn1.running_var", "layer2.3.bn1.num_batches_tracked", "layer2.3.conv2.weight", "layer2.3.bn2.weight", "layer2.3.bn2.bias", "layer2.3.bn2.running_mean", "

In [ ]:
parallelism = 2
model = models.resnet18(pretrained= False)
cur_model = models.resnet18(pretrained= False)

beta = 1.0/parallelism 
for i in range(parallelism):
    cur_model.load_state_dict(torch.load(f"./model_{i}.pt"))
    for key in cur_model.state_dict():
      if i == 0:
        model.state_dict()[key] = (cur_model.state_dict()[key]) / parallelism
      else:
        model.state_dict()[key] += (cur_model.state_dict()[key]) / parallelism
    #print(model)


print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch.nn as nn

# Setup
modelA = nn.Linear(3, 2)
modelB = nn.Linear(3, 2)
modelC = nn.Linear(3, 2)

print(modelA.state_dict())
print(modelB.state_dict())

sdA = modelA.state_dict()
sdB = modelB.state_dict()
sdC = modelC.state_dict()
# Average all parameters
for key in sdA:
    sdC[key] = (sdB[key] + sdA[key]) / 2.

# Recreate model and load averaged state_dict (or use modelA/B)
model = nn.Linear(3, 2)
model.load_state_dict(sdC)

print(model.state_dict())


for key in sdA:
    print(model.state_dict()[key] - (sdB[key] + sdA[key]) / 2)

OrderedDict([('weight', tensor([[ 0.3981,  0.3661, -0.4000],
        [-0.4682, -0.4987,  0.1842]])), ('bias', tensor([0.5153, 0.0663]))])
OrderedDict([('weight', tensor([[ 0.2693,  0.1602, -0.1152],
        [-0.4010,  0.0366, -0.1606]])), ('bias', tensor([-0.4497,  0.2997]))])
OrderedDict([('weight', tensor([[ 0.3337,  0.2631, -0.2576],
        [-0.4346, -0.2311,  0.0118]])), ('bias', tensor([0.0328, 0.1830]))])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [ ]:
import torch
import torch.nn as nn


modelA = nn.Linear(3, 2)
modelB = nn.Linear(3, 2)
modelC = nn.Linear(3, 2)


torch.save(modelA.state_dict(), f"model_0.pt")
torch.save(modelB.state_dict(), f"model_1.pt")
torch.save(modelC.state_dict(), f"model_2.pt")

sdA = modelA.state_dict()
sdB = modelB.state_dict()
sdC = modelC.state_dict()

sd_avg = modelB.state_dict()
parallelism = 3
model = nn.Linear(3, 2)
cur_model = nn.Linear(3, 2)

for i in range(parallelism):
    cur_model.load_state_dict(torch.load(f"./model_{i}.pt"))
    for key in cur_model.state_dict():
      if i == 0:
        sd_avg[key] = ((cur_model.state_dict()[key]) / parallelism)
      else:
        sd_avg[key] += (cur_model.state_dict()[key]) / parallelism
    #print(model)

model.load_state_dict(sd_avg)
for key in sdA:
    print(model.state_dict()[key] - (sdB[key] + sdA[key] + sdC[key]) / parallelism)

tensor([[-7.4506e-09,  3.7253e-09,  0.0000e+00],
        [ 0.0000e+00, -7.4506e-09,  0.0000e+00]])
tensor([ 4.6566e-09, -2.9802e-08])


In [25]:
def model_weight_average(parallelism):
    global loading_time
    model = models.resnet34(pretrained = False)
    cur_model = models.resnet34(pretrained = False)

    sd_avg = model.state_dict()


    for i in range(parallelism):
        startLoad = datetime.now()
        cur_model.load_state_dict(torch.load(f"./model_{i}.pt"))
        print('Model i: {}, loading time: {}'.format(i, (datetime.now() - startLoad).total_seconds()))
        loading_time += (datetime.now() - startLoad).total_seconds()
        for key in cur_model.state_dict():
            if i == 0:
                sd_avg[key] = (cur_model.state_dict()[key]) / parallelism
            else:
                sd_avg[key] += (cur_model.state_dict()[key]) / parallelism
    model.load_state_dict(sd_avg)
    return model

In [26]:
from datetime import datetime
from torchvision import models
import torch

loading_time = 0
start = datetime.now()
model = model_weight_average(4)
print("Total time is ", (datetime.now() - start).total_seconds())
print("Total loading is ", loading_time)

Model i: 0, loading time: 0.066617
Model i: 1, loading time: 0.09676
Model i: 2, loading time: 0.071199
Model i: 3, loading time: 0.102424
Total time is  1.293214
Total loading is  0.337267


In [30]:
start = datetime.now()
loading_time = 0

model = model_weight_average(4)
print("Total time is ", (datetime.now() - start).total_seconds())
print("Total loading is ", loading_time)

/home/ning/miniconda3/envs/nerf_ddp/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ning/miniconda3/envs/nerf_ddp/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model i: 0, loading time: 0.084807
Model i: 1, loading time: 0.206968
Model i: 2, loading time: 0.074931
Model i: 3, loading time: 0.101892
Total time is  1.444173
Total loading is  0.468866


In [33]:
from collections import defaultdict


In [38]:
def model_weight_average1(parallelism):
    global loading_time
    model = models.resnet34(pretrained = False)
    cur_model = models.resnet34(pretrained = False)

    sd_avg = defaultdict(list)


    for i in range(parallelism):
        startLoad = datetime.now()
        cur_model.load_state_dict(torch.load(f"./model_{i}.pt"))
        print('Model i: {}, loading time: {}'.format(i, (datetime.now() - startLoad).total_seconds()))
        loading_time += (datetime.now() - startLoad).total_seconds()
        for key in cur_model.state_dict():
            if i == 0:
                sd_avg[key] = (cur_model.state_dict()[key]) / parallelism
            else:
                sd_avg[key] += (cur_model.state_dict()[key]) / parallelism
    model.load_state_dict(sd_avg)
    return model

In [40]:
start = datetime.now()
loading_time = 0

model = model_weight_average1(4)
print("Total time is ", (datetime.now() - start).total_seconds())
print("Total loading is ", loading_time)

Model i: 0, loading time: 0.067198
Model i: 1, loading time: 0.117043
Model i: 2, loading time: 0.077684
Model i: 3, loading time: 0.106138
Total time is  1.376438
Total loading is  0.368351
